In [4]:
import polars as ps
import matplotlib.pyplot as plt

In [5]:
df = ps.read_csv("data/pacigeral_12_23.csv")
df.head()

ESCOLARI,IDADE,SEXO,UFNASC,UFRESID,IBGE,CIDADE,CATEATEND,DTCONSULT,CLINICA,DIAGPREV,DTDIAG,BASEDIAG,TOPO,TOPOGRUP,DESCTOPO,MORFO,DESCMORFO,EC,ECGRUP,T,N,M,PT,PN,PM,S,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,OUTRACLA,META01,META02,META03,META04,…,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,DTULTINFO,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,CICISUBGRU,FAIXAETAR,LATERALI,INSTORIG,DRS,RRAS,PERDASEG,ERRO,DTRECIDIVA,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN,CIDO,DSCCIDO,HABILIT,HABIT11,HABILIT1,HABILIT2,CIDADEH
i64,i64,i64,str,str,i64,str,i64,str,i64,i64,str,i64,str,str,str,i64,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,str,str,…,i64,i64,i64,i64,str,i64,i64,f64,f64,i64,str,str,str,str,i64,str,str,i64,i64,i64,str,i64,i64,i64,i64,str,str,str,str,i64,i64,str,i64,str,i64,i64,str
4,17,2,"""SP""","""SP""",3538709,"""PIRACICABA""",9,"""2011-06-21""",24,2,"""2011-06-09""",3,"""C499""","""C49""","""TECIDO CONJUNT…",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""",8,8,8,8,8,8,null,null,null,null,null,…,0,0,0,0,"""2018-08-24""",2,12,254.0,266.0,2011,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""10-19""",8,null,"""DRS 10 PIRACI…",14,1,0,null,1,0,0,0,null,null,null,null,3538709,80003,"""NEOPLASIA MALI…",9,"""UNACON com Ser…",1,1,"""Piracicaba"""
9,1,2,"""SP""","""SP""",3535507,"""PARAGUACU PAUL…",9,"""2005-02-25""",3,1,"""2005-02-25""",3,"""C310""","""C31""","""SEIO MAXILAR""",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""",8,8,8,8,8,8,null,"""C22""",null,null,null,…,0,0,0,0,"""2006-08-05""",4,0,0.0,0.0,2005,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""00-09""",8,null,"""DRS 09 MARÍLI…",10,0,0,null,1,0,0,0,null,null,null,null,3506003,80003,"""NEOPLASIA MALI…",14,"""Inativo""",6,5,"""Bauru"""
9,18,2,"""SP""","""SP""",3548500,"""SANTOS""",9,"""2012-05-14""",24,1,"""2012-05-16""",3,"""C494""","""C49""","""TEC CONJUNTIVO…",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""",null,null,null,8,8,8,8,8,8,null,null,null,null,null,…,0,0,0,0,"""2023-11-14""",2,2,2.0,0.0,2012,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""10-19""",1,null,"""DRS 04 SANTOS…",7,0,0,null,1,0,0,0,null,null,null,null,3548500,80003,"""NEOPLASIA MALI…",2,"""UNACON com Ser…",1,1,"""Santos"""
5,19,1,"""RR""","""SP""",3548500,"""SANTOS""",1,"""2017-08-10""",31,2,"""2016-12-03""",3,"""C444""","""C44""","""PELE DO COURO …",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""",8,8,8,8,8,8,"""T4 N0 M0""",null,null,null,null,…,0,0,0,0,"""2021-09-09""",2,250,20.0,270.0,2016,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""10-19""",8,null,"""DRS 04 SANTOS…",7,0,0,null,1,0,0,0,null,null,null,null,3548500,80003,"""NEOPLASIA MALI…",2,"""UNACON com Ser…",1,1,"""Santos"""
4,16,1,"""SP""","""SP""",3548500,"""SANTOS""",2,"""2020-04-17""",31,2,"""2020-04-17""",3,"""C310""","""C31""","""SEIO MAXILAR""",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""",8,8,8,8,8,8,"""EC4 PT4 PN2 PM…",null,null,null,null,…,0,0,0,0,"""2023-05-25""",2,0,287.0,287.0,2020,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""10-19""",1,null,"""DRS 04 SANTOS…",7,0,0,null,1,0,0,0,null,null,null,null,3548500,80003,"""NEOPLASIA MALI…",2,"""UNACON com Ser…",1,1,"""Santos"""


# Introdução
Seguindo o formato dos dados acima, iremos selecionar as colunas as quais mais fazem sentido para nosso sistema, e assim aplicar algum tipo de tratamento à elas

# Retirando colunas desnecessarias

Para um DataFrame menor e mais coeso com o que nossa aplicação irá utilizar, precisamos retirar certas colunas do nosso DF atual

In [ ]:
columns_to_remove = ["ESCOLARI", "IBGE", 
                     "UFNASC", "CIDADE", 
                     "DTCONSULT", "CLINICA", 
                     "TOPO", "TOPOGRUP",
                     "DESCTOPO", "MORFO",
                     "DESCMORFO", ""
                    ]

# Coluna sexo
para essa coluna, os dados estão em formato numérico (1- para sexo masculino, 2- para feminino), nessa etapa, faremos uma query para verificar se existe algum registro que não possui essa informação, caso esse seja o caso, iremos aplicar um método para preencher o registro com a informação. Em seguida, iremos transformar a informação numérica em uma string

In [6]:
df.filter(ps.col("SEXO").is_null() | ~((ps.col("SEXO") == 1) | (ps.col("SEXO") == 2)))

ESCOLARI,IDADE,SEXO,UFNASC,UFRESID,IBGE,CIDADE,CATEATEND,DTCONSULT,CLINICA,DIAGPREV,DTDIAG,BASEDIAG,TOPO,TOPOGRUP,DESCTOPO,MORFO,DESCMORFO,EC,ECGRUP,T,N,M,PT,PN,PM,S,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,OUTRACLA,META01,META02,META03,META04,…,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,DTULTINFO,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,CICISUBGRU,FAIXAETAR,LATERALI,INSTORIG,DRS,RRAS,PERDASEG,ERRO,DTRECIDIVA,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN,CIDO,DSCCIDO,HABILIT,HABIT11,HABILIT1,HABILIT2,CIDADEH
i64,i64,i64,str,str,i64,str,i64,str,i64,i64,str,i64,str,str,str,i64,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,str,str,…,i64,i64,i64,i64,str,i64,i64,f64,f64,i64,str,str,str,str,i64,str,str,i64,i64,i64,str,i64,i64,i64,i64,str,str,str,str,i64,i64,str,i64,str,i64,i64,str


Conforme o código acima, não existe nenhum dado que é nulo ou que seja diferente de 1 e 2, portanto, podemos prosseguir com o tratamento

In [7]:
df = df.with_columns(
    (ps.when(ps.col("SEXO") == 1)
     .then(ps.lit("Masculino"))
     .otherwise(ps.lit("Feminino"))
     .alias("SEXO")
    )
)
df.head()

ESCOLARI,IDADE,SEXO,UFNASC,UFRESID,IBGE,CIDADE,CATEATEND,DTCONSULT,CLINICA,DIAGPREV,DTDIAG,BASEDIAG,TOPO,TOPOGRUP,DESCTOPO,MORFO,DESCMORFO,EC,ECGRUP,T,N,M,PT,PN,PM,S,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,OUTRACLA,META01,META02,META03,META04,…,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,DTULTINFO,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,CICISUBGRU,FAIXAETAR,LATERALI,INSTORIG,DRS,RRAS,PERDASEG,ERRO,DTRECIDIVA,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN,CIDO,DSCCIDO,HABILIT,HABIT11,HABILIT1,HABILIT2,CIDADEH
i64,i64,str,str,str,i64,str,i64,str,i64,i64,str,i64,str,str,str,i64,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,str,str,…,i64,i64,i64,i64,str,i64,i64,f64,f64,i64,str,str,str,str,i64,str,str,i64,i64,i64,str,i64,i64,i64,i64,str,str,str,str,i64,i64,str,i64,str,i64,i64,str
4,17,"""Feminino""","""SP""","""SP""",3538709,"""PIRACICABA""",9,"""2011-06-21""",24,2,"""2011-06-09""",3,"""C499""","""C49""","""TECIDO CONJUNT…",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""",8,8,8,8,8,8,null,null,null,null,null,…,0,0,0,0,"""2018-08-24""",2,12,254.0,266.0,2011,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""10-19""",8,null,"""DRS 10 PIRACI…",14,1,0,null,1,0,0,0,null,null,null,null,3538709,80003,"""NEOPLASIA MALI…",9,"""UNACON com Ser…",1,1,"""Piracicaba"""
9,1,"""Feminino""","""SP""","""SP""",3535507,"""PARAGUACU PAUL…",9,"""2005-02-25""",3,1,"""2005-02-25""",3,"""C310""","""C31""","""SEIO MAXILAR""",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""",8,8,8,8,8,8,null,"""C22""",null,null,null,…,0,0,0,0,"""2006-08-05""",4,0,0.0,0.0,2005,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""00-09""",8,null,"""DRS 09 MARÍLI…",10,0,0,null,1,0,0,0,null,null,null,null,3506003,80003,"""NEOPLASIA MALI…",14,"""Inativo""",6,5,"""Bauru"""
9,18,"""Feminino""","""SP""","""SP""",3548500,"""SANTOS""",9,"""2012-05-14""",24,1,"""2012-05-16""",3,"""C494""","""C49""","""TEC CONJUNTIVO…",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""",null,null,null,8,8,8,8,8,8,null,null,null,null,null,…,0,0,0,0,"""2023-11-14""",2,2,2.0,0.0,2012,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""10-19""",1,null,"""DRS 04 SANTOS…",7,0,0,null,1,0,0,0,null,null,null,null,3548500,80003,"""NEOPLASIA MALI…",2,"""UNACON com Ser…",1,1,"""Santos"""
5,19,"""Masculino""","""RR""","""SP""",3548500,"""SANTOS""",1,"""2017-08-10""",31,2,"""2016-12-03""",3,"""C444""","""C44""","""PELE DO COURO …",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""",8,8,8,8,8,8,"""T4 N0 M0""",null,null,null,null,…,0,0,0,0,"""2021-09-09""",2,250,20.0,270.0,2016,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""10-19""",8,null,"""DRS 04 SANTOS…",7,0,0,null,1,0,0,0,null,null,null,null,3548500,80003,"""NEOPLASIA MALI…",2,"""UNACON com Ser…",1,1,"""Santos"""
4,16,"""Masculino""","""SP""","""SP""",3548500,"""SANTOS""",2,"""2020-04-17""",31,2,"""2020-04-17""",3,"""C310""","""C31""","""SEIO MAXILAR""",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""",8,8,8,8,8,8,"""EC4 PT4 PN2 PM…",null,null,null,null,…,0,0,0,0,"""2023-05-25""",2,0,287.0,287.0,2020,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""10-19""",1,null,"""DRS 04 SANTOS…",7,0,0,null,1,0,0,0,null,null,null,null,3548500,80003,"""NEOPLASIA MALI…",2,"""UNACON com Ser…",1,1,"""Santos"""


# Coluna Idade
Para esta coluna, devemos apenas fazer a verificação se há alguma observação com valor faltante

In [12]:
df.filter(ps.col("IDADE").is_null() | (ps.col("IDADE") <= 0))

ESCOLARI,IDADE,SEXO,UFNASC,UFRESID,IBGE,CIDADE,CATEATEND,DTCONSULT,CLINICA,DIAGPREV,DTDIAG,BASEDIAG,TOPO,TOPOGRUP,DESCTOPO,MORFO,DESCMORFO,EC,ECGRUP,T,N,M,PT,PN,PM,S,G,LOCALTNM,IDMITOTIC,PSA,GLEASON,OUTRACLA,META01,META02,META03,META04,…,HORMOAPOS,TMOAPOS,IMUNOAPOS,OUTROAPOS,DTULTINFO,ULTINFO,CONSDIAG,TRATCONS,DIAGTRAT,ANODIAG,CICI,CICIGRUP,CICISUBGRU,FAIXAETAR,LATERALI,INSTORIG,DRS,RRAS,PERDASEG,ERRO,DTRECIDIVA,RECNENHUM,RECLOCAL,RECREGIO,RECDIST,REC01,REC02,REC03,REC04,IBGEATEN,CIDO,DSCCIDO,HABILIT,HABIT11,HABILIT1,HABILIT2,CIDADEH
i64,i64,str,str,str,i64,str,i64,str,i64,i64,str,i64,str,str,str,i64,str,str,str,str,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,str,str,…,i64,i64,i64,i64,str,i64,i64,f64,f64,i64,str,str,str,str,i64,str,str,i64,i64,i64,str,i64,i64,i64,i64,str,str,str,str,i64,i64,str,i64,str,i64,i64,str
1,0,"""Feminino""","""SP""","""SP""",3547809,"""SANTO ANDRE""",2,"""2011-05-12""",25,1,"""2011-07-01""",3,"""C696""","""C69""","""ORBITA SOE""",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""",null,null,null,8,8,8,8,8,8,null,null,null,null,null,…,0,0,0,0,"""2023-02-13""",2,50,53.0,3.0,2011,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""00-09""",2,null,"""DRS 01 SÃO PA…",1,0,0,null,1,0,0,0,null,null,null,null,3547809,80003,"""NEOPLASIA MALI…",2,"""UNACON com Ser…",1,1,null
1,0,"""Masculino""","""SP""","""SP""",3522703,"""ITAPOLIS""",2,"""2014-05-12""",24,1,"""2014-05-15""",3,"""C493""","""C49""","""TEC CONJUNTIVO…",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""",null,null,null,8,8,8,8,8,8,null,null,null,null,null,…,0,0,0,0,"""2021-08-17""",2,3,3.0,0.0,2014,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""00-09""",8,null,"""DRS 03 ARARAQ…",13,1,0,null,1,0,0,0,null,null,null,null,3505500,80003,"""NEOPLASIA MALI…",7,"""CACON com Serv…",3,2,"""Barretos"""
9,0,"""Masculino""","""SP""","""SP""",3535507,"""PARAGUACU PAUL…",9,"""2011-02-25""",24,1,"""2011-02-25""",3,"""C762""","""C76""","""ABDOME SOE""",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""","""Y""",8,8,8,8,8,8,null,null,null,null,null,…,0,0,0,0,"""2013-04-12""",3,0,0.0,0.0,2011,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""00-09""",8,null,"""DRS 09 MARÍLI…",10,0,0,null,1,0,0,0,null,null,null,null,3529005,80003,"""NEOPLASIA MALI…",7,"""CACON com Serv…",3,2,"""Marilia"""
1,0,"""Feminino""","""SP""","""SP""",3553708,"""TAQUARITINGA""",2,"""2022-06-27""",28,2,"""2022-06-25""",3,"""C381""","""C38""","""MEDIASTINO ANT…",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""",null,null,null,8,8,8,8,8,8,null,null,null,null,null,…,0,0,0,0,"""2022-07-16""",3,2,11.0,13.0,2022,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""00-09""",8,null,"""DRS 03 ARARAQ…",13,0,0,null,1,0,0,0,null,null,null,null,3543402,80003,"""NEOPLASIA MALI…",7,"""CACON com Serv…",3,2,"""Ribeirão Preto…"
1,0,"""Feminino""","""SP""","""SP""",3550308,"""SAO PAULO""",2,"""2020-11-04""",25,1,"""2020-11-04""",3,"""C529""","""C52""","""VAGINA SOE""",80003,"""NEOPLASIA MALI…","""Y""","""Y""","""Y""","""Y""","""Y""",null,null,null,8,8,8,8,8,8,null,null,null,null,null,…,0,0,0,0,"""2021-08-18""",4,0,14.0,14.0,2020,"""XIIB""","""XII OUTRAS NE…","""B OUTROS TUMO…","""00-09""",8,null,"""DRS 01 SÃO PA…",6,0,0,null,1,0,0,0,null,null,null,null,3550308,80003,"""NEOPLASIA MALI…",5,"""UNACON exclusi…",2,1,"""São Paulo"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1,0,"""Feminino""","""SP""","""SP""",3550308,"""SAO PAULO""",1,"""2013-11-25""",25,2,"""2013-09-19""",3,"""C740""","""C74""","""GLANDULA SUPRA…",82313,"""CARCINOMA SIMP…","""Y""","""Y""","""Y""","""Y""","""Y""",null,null,null,8,8,8,8,8,8,null,null,null,null,null,…,0,0,0,0,"""2014-12-23""",1,67,0.0,67.0,2013,null,null,null,"""00-09""",2,null,"""DRS 01 SÃO PA…",6,1,0,null,1,0,0,0,null,null,null,null,3550308,82313,"""CARCINOMA SIMP…",7,"""CACON com Serv…",3

Com a analise acima, gera uma dúvida, essa coluna 